<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/02_BoW_TfIdf_Logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos entrenar sistemas de clasificación de texto con BoW/TF-IDF + Reg. Logística poniendo especial énfasis en los pasos más importantes que tenemos que seguir para resolver tareas de clasificación con Machine Learning.

---

TAREA: responder donde dice **PREGUNTA**

In [ ]:
%%capture
!pip install watermark datasets

In [ ]:
%load_ext watermark

In [ ]:
%watermark -udvp numpy,pandas,datasets,sklearn

### Datos

**IMPORTANTE**

* Siempre investigar un poco acerca del dataset antes de explorarlo (e.g. leer repositorios o papers asociados, etc.)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "emotion")

In [ ]:
dataset

**IMPORTANTE**

* Siempre explorar los datos para saber qué problemas puede haber (e.g. datos faltantes, labels inesperados en alguna partición del dataset, documentos muy largos, caracteres raros en el input, etc.)

In [ ]:
dataset["train"].features

In [ ]:
dataset["train"][0]

In [ ]:
# Convertimos los labels a strings por comodidad
int2label = dataset["train"].features["label"].int2str
dataset = dataset.map(lambda example: {"label_str": int2label(example["label"])}, remove_columns=["label"])
dataset = dataset.rename_column("label_str", "label")

In [ ]:
dataset["train"][0]

In [ ]:
# Convertimos a pandas por comodidad
import pandas as pd
pd.options.display.max_colwidth = 300

df_train = dataset["train"].to_pandas()
df_val = dataset["validation"].to_pandas()
df_test = dataset["test"].to_pandas()
#del dataset

In [ ]:
df_train.sample(10)

In [ ]:
for df in [df_train, df_val, df_test]:
    print(df["label"].value_counts(normalize=False))
    print()

**IMPORTANTE**

* El preprocesamiento del texto (las transformaciones que hagamos antes de correr modelos) depende del dominio de los datos, de las características del dataset particular, de los modelos vayamos a usar, de la tarea que queremos resolver, etc.
* Muchas veces lo mejor es evaluar si algún paso de preprocesamiento altera el rendimiento del modelo antes de aplicarlo, como vamos a ver más adelante

In [ ]:
# Vamos a hacer algo sencillo A MODO DE EJEMPLO.
# Convertimos 3 o mas letras repetidas a 3 letras: "hellooooo" -> "hellooo"
import re

re.sub(r"(\w)\1{2,}", r"\1\1\1", "hellooooo is there anyboooody in theere?")

In [ ]:
def preprocess_text(text):
    text = re.sub(r"(\w)\1{2,}", r"\1\1\1", text)
    text = text.strip()
    return text

In [ ]:
preprocess_text("hellooooo is there anyboooody in theere?")

In [ ]:
for df in [df_train, df_val, df_test]:
    df["text"] = df["text"].apply(preprocess_text)

**PREUNTA** ¿qué pasos de preprocesamiento aplicarían en esta tarea?

### Estrategia de evaluación

**IMPORTANTE**:

* Antes de definir qué modelos queremos probar y qué hiperparámetros queremos tunear, debemos definir cómo vamos a evaluar los modelos. En particular:

1. Métrica de evaluación
   * Tenemos un dataset de clasificación multiclase desbalanceado donde ninguna clase parece ser más importante que el resto
   * **PREGUNTA 1** ¿qué métrica podríamos usar?

2. Partición de datos (e.g. un único test set, holdout aka validation aka dev set, cross-validation)
   * El dataset ya viene con una partición train/dev/test
   * **PREGUNTA 2** ¿Para qué sirve cada partición?

### Modelos

**IMPORTANTE**:

* El _modelo_ no es solo el clasificador que corremos sobre los features, sino también la manera en la que generamos los features!
* Es fundamental entender qué hacen los modelos que vamos a probar, qué hiperparámetros tienen y qué hace cada uno -- si no entendemos los sistemas, es imposible saber por qué fallan y cómo mejorarlos.

In [ ]:
# Veamos cómo funciona BoW con CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer()

textos = [
    "Qué rico que es el mate",
    "el mate es muy... rico",
    "el mate es rico?",
]
# Aprende el vocabulario:
bow.fit(textos)

In [ ]:
# OJO: implícitamente hizo un preprocesamiento por default
print(bow.vocabulary_)

In [ ]:
# Para cada documento, contamos cuántas veces aparece cada palabra del vocabulario:
print(bow.transform(textos).toarray()) # transformamos cada doc en un vector de dimension fija
print(bow.get_feature_names_out())

In [ ]:
# Como DataFrame:
df_tmp = pd.DataFrame(bow.transform(textos).toarray(), columns=bow.get_feature_names_out())
df_tmp

In [ ]:
# Y si lo aplicamos a textos que no estaban en el entrenamiento?
textos_test = [
    "el mate es una bebida muy muy rica",
    "Aguante Boca",
    "Qué rico que es el té!!!",
]
df_tmp = pd.DataFrame(bow.transform(textos_test).toarray(), columns=bow.get_feature_names_out())
df_tmp

**IMPORTANTE**:

* Tener control de lo que estamos haciendo, es decir, entender exactamente cómo estamos representando los documentos.
    * **PREGUNTA 3** En el ejemplo anterior: ¿qué sucede con las palabras OOV? ¿Y con las palabras repetidas?

In [ ]:
# Corramos una version preliminar de un clasificador con BoW features:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score

vectorizer = CountVectorizer(max_features=5_000)
classifier = LogisticRegression(max_iter=1000, random_state=33)

vectorizer.fit(df_train["text"]) # Aprende el vocabulario
X_train = vectorizer.transform(df_train["text"]) # Transforma los textos en vectores
# equivalente:
# X_train = vectorizer.fit_transform(df_train["text"])

y_train = df_train["label"]
classifier.fit(X_train, y_train) # Entrena el clasificador


In [ ]:
# Evaluamos en el conjunto de validación:
X_val = vectorizer.transform(df_val["text"])
y_val = df_val["label"]
y_pred = classifier.predict(X_val)
print(fbeta_score(y_val, y_pred, beta=1, average="macro"))

**PREGUNTA 4** ¿Qué tipo de dato devuelve el método `.predict()`de este modelo? ¿Cómo se obtienen estos valores?

In [ ]:
# Con otros parametros:
vectorizer = CountVectorizer(stop_words="english", min_df=5)
classifier = LogisticRegression(max_iter=1000, random_state=33)

X_train = vectorizer.fit_transform(df_train["text"])
_ = classifier.fit(X_train, y_train)
X_val = vectorizer.transform(df_val["text"])
y_pred = classifier.predict(X_val)
print(fbeta_score(y_val, y_pred, beta=1, average="macro"))

In [ ]:
# Con TF-IDF:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5_000)
classifier = LogisticRegression(max_iter=1000, random_state=33)

X_train = vectorizer.fit_transform(df_train["text"])
print(vectorizer.idf_) # idf de cada palabra
print(vectorizer.get_feature_names_out()) # vocabulario

**PREGUNTA 5**: ¿por qué podría ser TF-IDF mejor que el BoW simple?

In [ ]:
_ = classifier.fit(X_train, y_train)
X_val = vectorizer.transform(df_val["text"])
y_pred = classifier.predict(X_val)
print(fbeta_score(y_val, y_pred, beta=1, average="macro"))

**IMPORTANTE**:

* ¿Cómo saber si el rendimiento es bueno? Dicho de otra manera: ¿estamos ganando algo con Machine Learning? --> para saberlo tenemos que comparar con un baseline!
* El baseline puede ser un modelo ingenuo que siempre predice la clase mayoritaria, predecir aleatoriamente, o algo basado en reglas de negocio, expresiones regulares, etc -- depende del caso


In [ ]:
# vamos usar un "dummy classifier" que predice segun las priors de entrenamiento
from sklearn.dummy import DummyClassifier

clf_dummy = DummyClassifier(strategy="prior")
_ = clf_dummy.fit(X_train, y_train)
y_pred = clf_dummy.predict(X_val)
print(fbeta_score(y_val, y_pred, beta=1, average="macro"))

# Con prediccion random?
clf_dummy = DummyClassifier(strategy="uniform")
_ = clf_dummy.fit(X_train, y_train)
y_pred = clf_dummy.predict(X_val)
print(fbeta_score(y_val, y_pred, beta=1, average="macro"))

**PREGUNTA 6** ¿qué quiere decir "predecir según las priors de entrenamiento" en el ejemplo de arriba?

#### Selección de modelos

**IMPORTANTE**:

* En general un modelo no es UN modelo, sino un montón de configuraciones posibles. ¿Qué cosas podemos variar en este caso?
  * stopwords (param `stop_words`)
  * binary features (param `binary`)
  * tokenización (param `tokenizer`, `nltk.tokenize.TweetTokenizer`, ...)
  * usar ngramas como tokens (param `ngram_range`)
  * preprocesamiento (param `preprocessor`: lowercase, stemming con `nltk.stem.SnowballStemmer`, eliminar palabras muy o poco frecuentes con `min_df` y `max_df`, etc.)
  * propagación de negaciones (https://arxiv.org/ftp/arxiv/papers/1305/1305.6143.pdf section 5)
  * features adicionales (e.g. conteo de palabras positivas/negativas, conteo de OOV words, ... usar la imaginacion!)
  * Hiperparámetros del clasificador (e.g. regularizacion)
  * **etc etc etc**

* Antes de ponerse a correr código, definir qué pruebas quiero hacer y a qué tipo de resultado quiero llegar (un número? una tabla? un gráfico?)

**PREGUNTA 7** ¿qué quiere decir "binary features" en este contexto?

In [ ]:
# para hacer seleccion de modelos de manera prolija podemos usar pipelines de sklearn
# (tambien lo podemos programar a mano, claro!)
from sklearn.pipeline import Pipeline

# Estos pasos se van a aplicar secuencialmente al input
pipe = Pipeline([
    ("vectorizer", TfidfVectorizer(max_features=5_000)),
    ("classifier", LogisticRegression(max_iter=1000, random_state=33)),
])

_ = pipe.fit(df_train["text"], y_train) # Ajusta todos los pasos
y_pred = pipe.predict(df_val["text"]) # Transforma y predice
print(fbeta_score(y_val, y_pred, beta=1, average="macro"))

# Para usar CV en lugar de val set, podemos usar cross_val_score:
# cross_val_score(pipe, X_train, y_train, cv=5, scoring='f1_macro')

In [ ]:
# A MODO ILUSTRATIVO, supongamos que nos interesa validar la eliminacion de
# stopwords, el uso de bigramas, y el uso de tf-idf:

# 1ro definimos un pipeline "esqueleto" con los pasos que queremos probar:
pipe = Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("classifier", LogisticRegression(max_iter=1000, random_state=33)),
])

In [ ]:
# 2do definimos el espacio de modelos que queremos explorar con una "grilla":
param_grid = {
    "vectorizer": [CountVectorizer(), TfidfVectorizer()],
    "vectorizer__stop_words": [None, "english"],
    "vectorizer__ngram_range": [(1, 1), (1, 2)],
    # "vectorizer__tokenizer": [...],
}
# Podemos separar pruebas usando una lista de diccionarios

In [ ]:
# 3ro entrenamos todas las configuraciones posibles en train y evaluamos cada una en val:
# (usamos PredefinedSplit para que el conjunto de validacion sea siempre el mismo en GridSearchCV)
from sklearn.model_selection import GridSearchCV, PredefinedSplit

X = pd.concat([df_train, df_val]).reset_index(drop=True)["text"]
y = pd.concat([df_train, df_val]).reset_index(drop=True)["label"]
val_fold = [-1]*len(df_train) + [0]*len(df_val)
ps = PredefinedSplit(val_fold)
grid_search = GridSearchCV(
    pipe, param_grid, cv=ps, scoring="f1_macro", refit=False, verbose=10)

# Si hay demasiadas configuraciones posibes, podemos usar RandomizedSearchCV

In [ ]:
_ = grid_search.fit(X, y)

**PREGUNTA 8** ¿por qué hay 8 lineas en el print anterior?

In [ ]:
df_results = pd.DataFrame(grid_search.cv_results_)
df_results.sort_values("rank_test_score").head(1)

In [ ]:
df_results.sort_values("rank_test_score").tail(1)
# El tuneo de HPs puede ser fundamental!!!

### Análisis de resultados

**IMPORTANTE**:

* Muchas veces no solo importa el rendimiento del modelo, sino también entender qué está haciendo. Por ejemplo:
  * Feature importance: A qué features le da importancia el modelo? Tiene sentido?
  * Análisis de errores: en qué casos falla? qué tipos de errores comete? Puede ser muy útil primero correr un modelo sencillo y analizar los errores que comete, para tener una idea de por dónde conviene seguir trabajando.
  * Usar la imaginación!

* A la hora de analizar y presentar métricas, poner el foco en las que nos importan -- no mostrar todos los números porque sí

In [ ]:
from sklearn.base import clone

best_pipe = clone(pipe)
best_pipe = best_pipe.set_params(**grid_search.best_params_)

In [ ]:
best_pipe

In [ ]:
# análisis de metricas
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

_ = best_pipe.fit(df_train["text"], df_train["label"])
y_pred = best_pipe.predict(df_val["text"])

print(classification_report(df_val["label"], y_pred))

cm = confusion_matrix(df_val["label"], y_pred)
plt.figure(figsize=(3,2))
sns.heatmap(
    cm, annot=True, fmt="d", cmap="Blues",
    xticklabels=best_pipe.classes_, yticklabels=best_pipe.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

**PREGUNTA 9**: ¿por qué cada clase tiene un valor de f-score distinto? ¿qué es el "macro avg"?

In [ ]:
# analisis de errores:
df_val["proba_pred"] = best_pipe.predict_proba(df_val["text"]).max(axis=1)
df_val["pred"] = y_pred
df_val["correct"] = df_val["label"] == df_val["pred"]

# Top 5 errores:
df_val[~df_val["correct"]].sort_values("proba_pred", ascending=False).head(5)

In [ ]:
# feature importance
features = best_pipe.named_steps["vectorizer"].get_feature_names_out()
classes = best_pipe.classes_
weights = best_pipe.named_steps["classifier"].coef_ # shape (n_classes, n_features)

for i, label in enumerate(classes):
    feat_importance = pd.Series(weights[i], index=features).sort_values(ascending=False)
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.barh(feat_importance.index[:10], feat_importance.values[:10], color="darkgreen")
    ax.barh(feat_importance.index[-10:], feat_importance.values[-10:], color="crimson")
    ax.invert_yaxis()
    plt.title(f"class={label}")
    plt.ylabel("Feature importance")
    plt.yticks(size=9)
    plt.show()


**PREGUNTA 10** ¿Qué está almacenado en el objeto `weights`? ¿Qué significa que sean negativos o positivos?


In [ ]:
# Evaluacion en test set

# cuando evaluamos en test podemos entrenar con train+val
X = pd.concat([df_train, df_val]).reset_index(drop=True)["text"]
y = pd.concat([df_train, df_val]).reset_index(drop=True)["label"]
_ = best_pipe.fit(X, y)

**PREGUNTA 11**: ¿por qué no hicimos la búsqueda de hiperparámetros en el test set?

In [ ]:
y_pred_test = best_pipe.predict(df_test["text"])

print(classification_report(df_test["label"], y_pred_test))

cm = confusion_matrix(df_test["label"], y_pred_test)
plt.figure(figsize=(3,2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=best_pipe.classes_, yticklabels=best_pipe.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

### Bonus track

¿Cómo agregar otros features?

In [ ]:
# Por ejemplo:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, FunctionTransformer
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler

X = pd.concat([df_train, df_val]).reset_index(drop=True)[["text"]]

X["ft_log_n_words"] = X["text"].apply(lambda x: np.log10(len(x.split())))
X["ft_log_n_users"] = np.log10(np.random.randint(1, 1_000, size=len(X))) # por ejemplo
X["ft_has_exclamation"] = X["text"].str.contains("!")
# OJO: acá podemos aplicar transformaciones a todos los datos juntos porque
# son transformaciones que no dependen de la particion train/val/test. Si no
# es así, podemos hacerlo en el pipeline para que no haya "data leakage".

# Esto puede estar un poco desactualizado pero anda,
# con versiones nuevas usar ColumnTransformer
class FeatSelector(BaseEstimator, TransformerMixin):
    def __init__(self, variables):
        self.variables = variables
    def fit(self, df, y=None):
        return self
    def transform(self, df):
        return df[self.variables]
    def get_feature_names(self):
        return self.variables

vectorizer = Pipeline([
    ('selector', FeatSelector(variables='text')),
    ('feat_extractor', TfidfVectorizer(min_df=10, max_features=1000, binary=True)),
    ('to_dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
])
other_features = ['ft_log_n_words', 'ft_log_n_users', 'ft_has_exclamation']

pipe = Pipeline([
    ('features', FeatureUnion([
        ('text', vectorizer),
        ('others', FeatSelector(variables=other_features)), # con "passthrough" podemos excluir este paso
    ])),
    ('scaler', MinMaxScaler()), # Fundamental si usamos regularizacion
    ('clf', LogisticRegression(max_iter=1000, random_state=33)),
])

X_train = X.loc[df_train.index]
y_train = df_train["label"]
X_val = X.loc[df_val.index]

_ = pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_val)
print(fbeta_score(df_val["label"], y_pred, beta=1, average="macro"))

---------------------------------------